# Code to Integrate Fundamental Data from Screener.in and Pricing Data from Yahoo Finance

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')

# Read the Screener Data for Fundamental Data integration with Market Data

In [14]:
yearly_data=pd.read_csv('data/Yearly_Ticker_nifty500_standalone.csv')
yearly_data.head()
ticker_list=yearly_data['Ticker'].unique().tolist()
type_list=yearly_data['Type'].unique().tolist()
for types in type_list:
    z3=pd.DataFrame()
    file_name=types+'.csv'
    for ticker in ticker_list:
        z=yearly_data[yearly_data['Type']==types]
        z1=z[z['Ticker']==ticker]
        z2=z1.drop(['Ticker','Type'],axis=1).T
        z2.columns = z2.iloc[0]
        z2=z2.drop(z2.index[0])
        z2['Symbol']=ticker
        z3=z3.append(z2)
    #z3.rename(columns={ z3.columns[1]: "Date" }, inplace = True)
    z3.columns = z3.columns.str.replace("[+]", "")
    z3.columns = z3.columns.str.replace("[%]", "")
    z3.columns = z3.columns.str.replace("[Â]", "")
    z3.to_csv(file_name)
final=pd.DataFrame()
income_s=pd.read_csv('Income_Statement.csv')
income_s.rename(columns = {'Unnamed: 0':'Date'}, inplace = True) 
income_s=income_s.fillna(0)
income_s['Sales']=income_s['Sales'].astype(float)
#income_s['Revenue']=income_s['Revenue'].astype(float)
#income_s['Sales1'] = income_s['Sales']+ income_s['Revenue']
#income_s['Sales1']=income_s['Sales1'].replace('nan','',regex=True).astype(float)
#income_s['Sales']=income_s['Sales1']
#income_s=income_s.drop(['Sales1','Revenue','Financing Profit','Financing Margin '],axis=1)
balance_s=pd.read_csv('Balance_Sheet.csv')
balance_s.rename(columns = {'Unnamed: 0':'Date'}, inplace = True) 
cash_s=pd.read_csv('Cash_Flow.csv')
cash_s.rename(columns = {'Unnamed: 0':'Date'}, inplace = True) 
final_y=income_s.merge(balance_s,how='inner',on=['Date','Symbol']).merge(cash_s,how='inner',on=['Date','Symbol'])
final_y.columns = final_y.columns.str.replace(' ', '')
final_y_1=final_y[['Date','Symbol','Sales','OPM','NetProfit','ShareCapital','EPSinRs','DividendPayout','OtherLiabilities','TotalAssets','NetCashFlow']]
#final_y_1['Date'] = pd.to_datetime(pd.DatetimeIndex(pd.to_datetime(final_y_1['Date'], format='%Y')).year,format='%Y')
final_y_1['Trail_EPSinRs']=final_y_1.groupby('Symbol')['EPSinRs'].shift(1)
final_y_1['Trail_Sales']=final_y_1.groupby('Symbol')['Sales'].shift(1)
final_y_1['Date1']=final_y_1['Date']
#final_y_1=final_y_1.drop_duplicates()

# Backfilling the details for missing data 

In [15]:
column_list=['Sales','OPM','NetProfit','ShareCapital','EPSinRs','DividendPayout','OtherLiabilities','TotalAssets','NetCashFlow','Trail_EPSinRs','Trail_Sales']
for column in column_list:
    #print("Column Name is:", column)
    final_y_1[final_y_1[column]==""] = np.NaN
    final_y_1[column]=final_y_1.groupby('Symbol')[column].fillna(method='bfill')
    final_y_1[column]=final_y_1.groupby('Symbol')[column].fillna(method='ffill')

# Calculate Shares Outstanding

In [16]:
symbol_list=final_y_1['Symbol'].unique().tolist()
final_y_2=pd.DataFrame()
for symbol in symbol_list:
    temp=final_y_1[final_y_1['Symbol']==symbol]
    temp['Open_Shares_old']=abs(temp['NetProfit']/temp['EPSinRs'])
    median=temp['Open_Shares_old'].median()
    temp["Open_Shares"] = np.where(temp["Open_Shares_old"] > 1.2*median, median, temp["Open_Shares_old"])
    temp["Open_Shares"] = np.where(temp["Open_Shares_old"] < 0.8*median, median, temp["Open_Shares_old"])
    final_y_2=final_y_2.append(temp)

In [17]:
final_y_2['Sales_Growth']=final_y_2.groupby('Symbol')['Sales'].apply(lambda x:x/x.shift(1)-1)
final_y_2['Sales_Growth']=final_y_2.groupby('Symbol')['Sales_Growth'].fillna(method='bfill')
final_y_2['Sales_Growth']=final_y_2.groupby('Symbol')['Sales_Growth'].fillna(method='ffill')
final_y_2['Sales_Growth']=final_y_2['Sales_Growth'].replace([np.inf, -np.inf], 0)
final_y_2['EPS_Growth']=final_y_2.groupby('Symbol')['EPSinRs'].apply(lambda x:x/x.shift(1)-1)
final_y_2['EPS_Growth']=final_y_2.groupby('Symbol')['EPS_Growth'].fillna(method='bfill')
final_y_2['EPS_Growth']=final_y_2.groupby('Symbol')['EPS_Growth'].fillna(method='ffill')
final_y_2['EPS_Growth']=final_y_2['EPS_Growth'].replace([np.inf, -np.inf], 0)
final_y_2['OPM'] = final_y_2['OPM'].str.replace("[%]", "")
final_y_2['OPM']=pd.to_numeric(final_y_2['OPM'])
final_y_2['OPM_Growth']=final_y_2.groupby('Symbol')['OPM'].apply(lambda x:x/x.shift(1)-1)
final_y_2['OPM_Growth']=final_y_2.groupby('Symbol')['OPM_Growth'].fillna(method='bfill')
final_y_2['OPM_Growth']=final_y_2.groupby('Symbol')['OPM_Growth'].fillna(method='ffill')
final_y_2['OPM_Growth']=final_y_2['OPM_Growth'].replace([np.inf, -np.inf], 0)
final_y_2['ROE']=final_y_2['Trail_EPSinRs']/final_y_2['ShareCapital']
final_y_2['ROE']=final_y_2.groupby('Symbol')['ROE'].fillna(method='bfill')
final_y_2['ROE']=final_y_2.groupby('Symbol')['ROE'].fillna(method='ffill')
final_y_2['ROE']=final_y_2['ROE'].replace([np.inf, -np.inf], 0)
final_y_2['Leverage']=final_y_2['OtherLiabilities']/final_y_2['ShareCapital']
final_y_2['Leverage']=final_y_2.groupby('Symbol')['Leverage'].fillna(method='bfill')
final_y_2['Leverage']=final_y_2.groupby('Symbol')['Leverage'].fillna(method='ffill')
final_y_2['Leverage']=final_y_2['Leverage'].replace([np.inf, -np.inf], 0)
final_y_2['Quality']=(final_y_2['ROE']+final_y_2['Leverage'])/2
final_y_2['Leverage']=final_y_2['OtherLiabilities']/final_y_2['ShareCapital']
final_y_2['Leverage']=final_y_2.groupby('Symbol')['Leverage'].fillna(method='bfill')
final_y_2['Leverage']=final_y_2.groupby('Symbol')['Leverage'].fillna(method='ffill')
final_y_2['Leverage']=final_y_2['Leverage'].replace([np.inf, -np.inf], 0)
final_y_2['BVPS']=final_y_2['ShareCapital']/final_y_2['Open_Shares']
final_y_2['BVPS']=final_y_2.groupby('Symbol')['BVPS'].fillna(method='bfill')
final_y_2['BVPS']=final_y_2.groupby('Symbol')['BVPS'].fillna(method='ffill')
final_y_2['BVPS']=final_y_2['BVPS'].replace([np.inf, -np.inf], 0)
final_y_2['Sale_per_share']=final_y_2['Sales']/final_y_2['Open_Shares']
final_y_2['Sale_per_share']=final_y_2.groupby('Symbol')['Sale_per_share'].fillna(method='bfill')
final_y_2['Sale_per_share']=final_y_2.groupby('Symbol')['Sale_per_share'].fillna(method='ffill')
final_y_2['Sale_per_share']=final_y_2['Sale_per_share'].replace([np.inf, -np.inf], 0)
final_y_2['Value']=(final_y_2['BVPS']+final_y_2['Sale_per_share']+final_y_2['EPSinRs'])/3
final_y_2['DividendPayout'] = final_y_2['DividendPayout'].str.replace("[%]", "")
final_y_2['DividendPayout']=pd.to_numeric(final_y_2['DividendPayout'])

In [18]:
final_y_2['Date1']=pd.to_datetime(final_y_1['Date'], format='%Y')
final_y_2['Date1']=pd.to_datetime(pd.DatetimeIndex(final_y_2['Date1']).year)

In [19]:
concat_df=pd.read_csv('data/NSE_500_Output.csv')
concat_df['Date']= pd.to_datetime(concat_df['Date'])
concat_df['Date1'] = pd.to_datetime(pd.DatetimeIndex(concat_df['Date']).year)

In [20]:
concat_df=concat_df.merge(final_y_2[['Symbol','Date1','Sales','OPM', 'NetProfit','EPSinRs', 'DividendPayout', 'OtherLiabilities', 
       'Open_Shares','Sales_Growth', 'EPS_Growth','OPM_Growth', 'ROE', 'Leverage', 'BVPS', 'Sale_per_share']].dropna(),how='left',on=['Symbol','Date1'])

In [21]:
concat_df['Sales_Growth']=concat_df.groupby('Symbol')['Sales_Growth'].fillna(method='ffill')
concat_df['Sales_Growth']=concat_df.groupby('Symbol')['Sales_Growth'].apply(lambda x:x.interpolate(method='linear', limit_direction='backward', axis=0))
concat_df['EPS_Growth']=concat_df.groupby('Symbol')['EPS_Growth'].fillna(method='ffill')
concat_df['EPS_Growth']=concat_df.groupby('Symbol')['EPS_Growth'].apply(lambda x:x.interpolate(method='linear', limit_direction='backward', axis=0))
concat_df['EPS in Rs']=concat_df.groupby('Symbol')['EPSinRs'].fillna(method='ffill')
concat_df['EPS']=concat_df.groupby('Symbol')['EPSinRs'].apply(lambda x:x.interpolate(method='linear', limit_direction='backward', axis=0))
concat_df['Sales']=concat_df.groupby('Symbol')['Sales'].fillna(method='ffill')
concat_df['Sales']=concat_df.groupby('Symbol')['Sales'].apply(lambda x:x.interpolate(method='linear', limit_direction='backward', axis=0))
concat_df['NetProfit']=concat_df.groupby('Symbol')['NetProfit'].fillna(method='ffill')
concat_df['NetProfit']=concat_df.groupby('Symbol')['NetProfit'].apply(lambda x:x.interpolate(method='linear', limit_direction='backward', axis=0))
concat_df['Open_Shares']=concat_df.groupby('Symbol')['Open_Shares'].fillna(method='ffill')
concat_df['Open_Shares']=concat_df.groupby('Symbol')['Open_Shares'].apply(lambda x:x.interpolate(method='linear', limit_direction='backward', axis=0))
concat_df['OPM_Growth']=concat_df.groupby('Symbol')['OPM_Growth'].fillna(method='ffill')
concat_df['OPM_Growth']=concat_df.groupby('Symbol')['OPM_Growth'].apply(lambda x:x.interpolate(method='linear', limit_direction='backward', axis=0))
concat_df['ROE']=concat_df.groupby('Symbol')['ROE'].fillna(method='ffill')
concat_df['ROE']=concat_df.groupby('Symbol')['ROE'].apply(lambda x:x.interpolate(method='linear', limit_direction='backward', axis=0))
concat_df['Leverage']=concat_df.groupby('Symbol')['Leverage'].fillna(method='ffill')
concat_df['Leverage']=concat_df.groupby('Symbol')['Leverage'].apply(lambda x:x.interpolate(method='linear', limit_direction='backward', axis=0))
concat_df['BVPS']=concat_df.groupby('Symbol')['BVPS'].fillna(method='ffill')
concat_df['BVPS']=concat_df.groupby('Symbol')['BVPS'].apply(lambda x:x.interpolate(method='linear', limit_direction='backward', axis=0))
concat_df['Sale_per_share']=concat_df.groupby('Symbol')['Sales'].fillna(method='ffill')
concat_df['Sale_per_share']=concat_df.groupby('Symbol')['Sales'].apply(lambda x:x.interpolate(method='linear', limit_direction='backward', axis=0))

# Calculate Derived Factors like E/P, BV/Price, Fama French Size Factor

In [22]:
concat_df['E/P']=concat_df['EPS']/concat_df['Adj Close']
concat_df['Sales/Price']=concat_df['Sale_per_share']/concat_df['Adj Close']
concat_df['BV/Price']=concat_df['BVPS']/concat_df['Adj Close']
concat_df['PEG']=1/(concat_df['EPS_Growth']*concat_df['E/P'])
#final_y_2['DividendPayout'] = final_y_2['DividendPayout'].str.replace("[%]", "")
concat_df['Div/Price']=concat_df['DividendPayout']/concat_df['Adj Close']
#concat_df['EPC_NP']=concat_df['EPS']/concat_df['NetProfit']
#concat_df['PES']=concat_df['Adj Close']*concat_df['Open_Shares']/concat_df['Sales']
concat_df['Size_Factor']=-1*np.log(concat_df['Adj Close']*concat_df['Open_Shares'])

In [23]:
concat_df['E/P']=concat_df['E/P'].replace([np.inf, -np.inf], 0)
concat_df['Sales/Price']=concat_df['Sales/Price'].replace([np.inf, -np.inf], 0)
concat_df['BV/Price']=concat_df['BV/Price'].replace([np.inf, -np.inf], 0)
concat_df['PEG']=concat_df['PEG'].replace([np.inf, -np.inf], 0)
concat_df['Div/Price']=concat_df['Sales/Price'].replace([np.inf, -np.inf], 0)

# Calculate Quality and Value Factors

In [24]:
column_list=['Volume','Share_Turnover', 'Return', 'Momentum', 'StDev',
       'Liquidity', 'Volatility','Sales', 'OPM',
       'NetProfit', 'EPSinRs', 'DividendPayout', 'OtherLiabilities',
       'Open_Shares', 'Sales_Growth', 'EPS_Growth', 'OPM_Growth', 'ROE',
       'Leverage', 'BVPS', 'Sale_per_share', 'EPS in Rs', 'EPS', 'E/P',
       'Sales/Price', 'BV/Price', 'PEG', 'Size_Factor']
for column in column_list:
    concat_df[concat_df[column]==""] = np.NaN
    concat_df[column]=concat_df.groupby('Symbol')[column].fillna(method='bfill')
    concat_df[column]=concat_df.groupby('Symbol')[column].fillna(method='ffill')
concat_df=concat_df.drop(['Unnamed: 0', 'Date1'],axis=1)
concat_df['StDev']=1/concat_df['StDev']
concat_df['Quality']=(concat_df['ROE']+concat_df['Leverage'])/2
concat_df['Value']=(concat_df['E/P']+concat_df['Sales/Price']+concat_df['BV/Price'])/3

In [25]:
concat_df.to_csv('data/Data_Y_nifty500.csv')